In [1]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [33]:
import collect_stats as cs

In [23]:
filenames = [
    'HOXC4_R158L_R1_8mers.txt',
    'HOXD13_REF_R1_8mers.txt',
    'GFI1_REF_R1_8mers.txt',
    'HOXC4_R158L_R2_8mers.txt',
    'PAX4_REF_R1_8mers.txt',
    'OVOL2_D228E_R1_8mers.txt',
    'POU3F4_V322M_R1_8mers.txt',
    'KLF11_R402Q_R1_8mers.txt',
    'HOXB7_R189C_R1_8mers.txt',
    'CRX_R90W_R1_8mers.txt',
]

In [ ]:
mves_condense_compare_w_old = cs.get_data_mves('/cluster/sj1/bb_opt', 'e')

In [5]:
mves = cs.get_data_mves('/cluster/sj1/bb_opt', 'n')

reading /cluster/sj1/bb_opt/output_n1/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n1/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n2/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n3/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n4/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/HOXD13_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n5/GFI1_REF_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n6/HOXC4_R158L_R1_8mers.txt
reading /cluster/sj1/bb_opt/output_n6/HOXD13_REF_R1_8mers.txt
reading /c

In [40]:
mves_condense = cs.get_data_mves('/cluster/sj1/bb_opt', 'f')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [41]:
mves_ei_mix = cs.get_data_mves('/cluster/sj1/bb_opt', 'ei_mves_mix')

reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10


In [42]:
ucb = cs.get_data_ucb('/cluster/sj1/bb_opt')

reading 0
read 10 samples
reading 1
read 10 samples
reading 2
read 10 samples
reading 3
read 10 samples
reading 4
read 8 samples


In [45]:
batch_idx = 3
ack_iter_idx = 19
stat_idx = -3

In [46]:
for f in filenames:
    print(f)
    try:
        arr = mves_ei_mix[f][batch_idx][:, ack_iter_idx, stat_idx]
        print('mves_ei_mix:', mves_ei_mix[f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr))
    except Exception as e:
        pass
    
    try:
        arr = mves_condense[f][batch_idx][:, ack_iter_idx, stat_idx]
        print('mves_condense:', mves_condense[f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr))
    except Exception as e:
        pass
    
    ucb_coeffs = [0.0, 0.5, 1.0, 2.0, 3.0]
    for i in range(5):
        try:
            if len(ucb[i][f][batch_idx]) != 0:
                arr = ucb[i][f][batch_idx][:, ack_iter_idx, stat_idx]
                print('ucb:', ucb_coeffs[i], ucb[i][f][batch_idx].shape[0], np.mean(arr), np.std(arr), np.median(arr))
        except Exception as e:
            pass

HOXC4_R158L_R1_8mers.txt
mves_ei_mix: 10 0.16423358 0.019441897 0.17001218
mves_condense: 10 0.13375913 0.009456438 0.1335158
ucb: 0.0 10 0.12761557 0.0063260347 0.12591241
ucb: 0.5 10 0.12214111 0.0045745997 0.123783454
ucb: 1.0 10 0.12682481 0.008354616 0.12712896
ucb: 2.0 10 0.12487835 0.0068443497 0.12469587
ucb: 3.0 8 0.11777677 0.007572549 0.11709246
HOXD13_REF_R1_8mers.txt
mves_ei_mix: 10 0.17086375 0.019860726 0.17883211
mves_condense: 10 0.13059612 0.01566476 0.13412409
ucb: 0.0 10 0.12329684 0.007269048 0.12621655
ucb: 0.5 10 0.12049878 0.005683038 0.11739659
ucb: 1.0 10 0.123236015 0.0071879174 0.123783454
ucb: 2.0 10 0.12445257 0.0057525975 0.125
ucb: 3.0 8 0.11861314 0.005865968 0.117700726
GFI1_REF_R1_8mers.txt
mves_ei_mix: 10 0.1276764 0.022326916 0.13625303
mves_condense: 10 0.09635036 0.014740289 0.09854014
ucb: 0.0 10 0.09166666 0.008419907 0.09184915
ucb: 0.5 10 0.08643552 0.008108297 0.08576642
ucb: 1.0 10 0.08613139 0.0078427065 0.08424574
ucb: 2.0 10 0.08941605 0.